In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm
import requests, re

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )


a = datetime.now()-timedelta(days=48)
a = a.strftime('%Y-%m-%d')
b = datetime.now()-timedelta(days=56)
b = b.strftime('%Y-%m-%d')
print(b, a)

2021-04-07 2021-04-15


In [3]:
print("=" *80)
print("         FM 정치/시사 게시판 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\daily\\{b}-{a}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         FM 정치/시사 게시판 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'fm.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
url2 = []       # 게시물 url
time2 = []      # 게시물 작성 시간
view_cnt2 = []  # 게시물 뷰 카운트
text2 = []      # 게시물 내용
com_cnt2 = []   # 댓글 수
com_user2 = []  # 작성자 본인을 제외한 댓글 유저
com_text2 = []  # 댓글 user_id & text

In [38]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
domain1 = "fm"
print("==========",domain1,"크롤링 진행 중 ==========")

========== fm 크롤링 진행 중 ==========


In [42]:
# page_num = 6609
while_code = 0

while True :
    if while_code == 1 :
        print("1차 데이터 수집 완료")
        print("2차 데이터 수집으로 넘어갑니다.")
        break
    
    driver.get(f'https://www.fmkorea.com/index.php?mid=news&page={page_num}')
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div/div/div/div/div/div/table/thead")))        
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    post_box = soup.find('tbody').find_all('tr', class_=False)
    
    for post in post_box :
        
        post_url = "https://www.fmkorea.com/" + post.find("td", class_=re.compile('title')).find('a')['href']
        print("url :", post_url)
#         req2 = requests.get(post_url)
        driver.get(post_url)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#bd_capture")))        
        time.sleep(1)
        html2 = driver.page_source
#         html2 = req2.text
        soup2 = BeautifulSoup(html2, 'lxml')
#         print(soup2.select('#bd_4180795_3604166605 > div.rd.rd_nav_style2.clear'))
#         print("aa:", soup2.find('span', class_='date m_no'))
#         print("bb:", soup2.find('span', class_='date m_no').text)
        post_date = soup2.find('span', class_='date m_no').text[:10].replace(".","-")
        
        if post_date > a :
#             page_num += 1
            print(post_date, " > ", a)
            print("페이지를 넘깁니다.\n")
            break
            
        elif post_date >= b :
            if post_url not in url2 :

                domain1 = domain1
                title1 = soup2.find('span', class_='np_18px_span').text
                user1 = soup2.find('a', class_=re.compile('nick member')).text
                url1 = post_url
                view1 = soup2.find_all(class_='side fr')[0].find('b').text
                try :
                    com_cnt1 = soup2.find(class_='ui_font bubble').find('b').text
                except :
                    com_cnt1 = 0

                text1 = soup2.find('article').text
                com_user1 = []
                com_text1 = []

                try :

                    com_box = soup2.find(class_='fdb_lst_ul').find_all('li')

                    for com in com_box :

                        com_user = com.find('a', class_=re.compile('member_')).text

                        try :
                            delete = com.find('a', class_='findParent').text
                            comment = com.find('div', class_=re.compile('comment_')).text
                            comment.replace(delete, "")
                        except :
                            comment = com.find('div', class_=re.compile('comment_')).text

                        com_user1.append(com_user)
                        com_text1.append(f"작성자 : {com_user} :: {comment}")
                except :
                    com_user1.append("")
                    com_text1.append("")       

    #             print("게시일 :", post_date)
    #             print("제목 :", title1)
    #             print("작성자 :", user1)
    #             print("url :", url1)
    #             print("조회수 :", view1)
    #             print("게시물 내용 :", text1)
    #             print("댓글수 :", com_cnt1)
    #             print("댓글 작성자 :", com_user1)
    #             print("댓글 내용 :", com_text1)
    #             print("\n")

                domain2.append(domain1)
                title2.append(title1)
                time2.append(post_date)
                user2.append(user1)
                url2.append(url1)
                view_cnt2.append(view1)
                com_cnt2.append(com_cnt1)
                text2.append(text1)
                com_user2.append(com_user1)
                com_text2.append(com_text1)

    #             time.sleep(2)

            
        else :
            while_code = 1
            break

    page_num += 1
    driver.delete_all_cookies()


url : https://www.fmkorea.com//3521919381
url : https://www.fmkorea.com//3521929474
url : https://www.fmkorea.com//3521929312
url : https://www.fmkorea.com//3521928402
url : https://www.fmkorea.com//3521928616
url : https://www.fmkorea.com//3521926867
url : https://www.fmkorea.com//3521916850
url : https://www.fmkorea.com//3521925264
url : https://www.fmkorea.com//3521925220
url : https://www.fmkorea.com//3521924685
url : https://www.fmkorea.com//3521924675
url : https://www.fmkorea.com//3521924164
url : https://www.fmkorea.com//3521923797
url : https://www.fmkorea.com//3521923360
url : https://www.fmkorea.com//3521921219
url : https://www.fmkorea.com//3521921542
url : https://www.fmkorea.com//3521912143
url : https://www.fmkorea.com//3521920932
url : https://www.fmkorea.com//3521919797
url : https://www.fmkorea.com//3521918648
url : https://www.fmkorea.com//3521912143
url : https://www.fmkorea.com//3521920932
url : https://www.fmkorea.com//3521919797
url : https://www.fmkorea.com//352

url : https://www.fmkorea.com//3521829796
url : https://www.fmkorea.com//3521829168
url : https://www.fmkorea.com//3521828670
url : https://www.fmkorea.com//3521827955
url : https://www.fmkorea.com//3521828306
url : https://www.fmkorea.com//3521828292
url : https://www.fmkorea.com//3521826842
url : https://www.fmkorea.com//3521827269
url : https://www.fmkorea.com//3521827219
url : https://www.fmkorea.com//3521827148
url : https://www.fmkorea.com//3521826249
url : https://www.fmkorea.com//3521818814
url : https://www.fmkorea.com//3521825026
url : https://www.fmkorea.com//3521824976
url : https://www.fmkorea.com//3521824948
url : https://www.fmkorea.com//3521824321
url : https://www.fmkorea.com//3521821724
url : https://www.fmkorea.com//3521823440
url : https://www.fmkorea.com//3521823170
url : https://www.fmkorea.com//3521823032
url : https://www.fmkorea.com//3521822157
url : https://www.fmkorea.com//3521822795
url : https://www.fmkorea.com//3521822524
url : https://www.fmkorea.com//352

url : https://www.fmkorea.com//3521729232
url : https://www.fmkorea.com//3521729137
url : https://www.fmkorea.com//3521728842
url : https://www.fmkorea.com//3521728407
url : https://www.fmkorea.com//3521728356
url : https://www.fmkorea.com//3521727774
url : https://www.fmkorea.com//3521724079
url : https://www.fmkorea.com//3521724829
url : https://www.fmkorea.com//3521728356
url : https://www.fmkorea.com//3521727774
url : https://www.fmkorea.com//3521724079
url : https://www.fmkorea.com//3521724829
url : https://www.fmkorea.com//3521724648
url : https://www.fmkorea.com//3521724699
url : https://www.fmkorea.com//3521719698
url : https://www.fmkorea.com//3521724512
url : https://www.fmkorea.com//3521722349
url : https://www.fmkorea.com//3521722043
url : https://www.fmkorea.com//3521719849
url : https://www.fmkorea.com//3521718919
url : https://www.fmkorea.com//3521718478
url : https://www.fmkorea.com//3521717529
url : https://www.fmkorea.com//3521717219
url : https://www.fmkorea.com//352

url : https://www.fmkorea.com//3521647817
url : https://www.fmkorea.com//3521648275
url : https://www.fmkorea.com//3521648117
url : https://www.fmkorea.com//3521647466
url : https://www.fmkorea.com//3521647235
url : https://www.fmkorea.com//3521646734
url : https://www.fmkorea.com//3521644466
url : https://www.fmkorea.com//3521645117
url : https://www.fmkorea.com//3521644499
url : https://www.fmkorea.com//3521644026
url : https://www.fmkorea.com//3521643276
url : https://www.fmkorea.com//3521643166
url : https://www.fmkorea.com//3521645117
url : https://www.fmkorea.com//3521644499
url : https://www.fmkorea.com//3521644026
url : https://www.fmkorea.com//3521643276
url : https://www.fmkorea.com//3521643166
url : https://www.fmkorea.com//3521640805
url : https://www.fmkorea.com//3521642074
url : https://www.fmkorea.com//3521641662
url : https://www.fmkorea.com//3521639969
url : https://www.fmkorea.com//3521641479
url : https://www.fmkorea.com//3521640290
url : https://www.fmkorea.com//352

url : https://www.fmkorea.com//3521558311
url : https://www.fmkorea.com//3521558872
url : https://www.fmkorea.com//3521518963
url : https://www.fmkorea.com//3521557480
url : https://www.fmkorea.com//3521558425
url : https://www.fmkorea.com//3521558173
url : https://www.fmkorea.com//3521558159
url : https://www.fmkorea.com//3521554985
url : https://www.fmkorea.com//3521557154
url : https://www.fmkorea.com//3521556853
url : https://www.fmkorea.com//3521555889
url : https://www.fmkorea.com//3521555710
url : https://www.fmkorea.com//3521554996
url : https://www.fmkorea.com//3521554113
url : https://www.fmkorea.com//3521553347
url : https://www.fmkorea.com//3521553151
url : https://www.fmkorea.com//3521552934
url : https://www.fmkorea.com//3521552861
url : https://www.fmkorea.com//3521550980
url : https://www.fmkorea.com//3521551264
url : https://www.fmkorea.com//3521551793
url : https://www.fmkorea.com//3521551513
url : https://www.fmkorea.com//3521551168
url : https://www.fmkorea.com//352

...

url : https://www.fmkorea.com//3507066058
url : https://www.fmkorea.com//3507065936
url : https://www.fmkorea.com//3507065473
url : https://www.fmkorea.com//3507065709
url : https://www.fmkorea.com//3507065284
url : https://www.fmkorea.com//3507065206
url : https://www.fmkorea.com//3507065201
url : https://www.fmkorea.com//3507064964
url : https://www.fmkorea.com//3507064848
url : https://www.fmkorea.com//3507064540
url : https://www.fmkorea.com//3507064441
url : https://www.fmkorea.com//3507064212
url : https://www.fmkorea.com//3507063850
url : https://www.fmkorea.com//3507062584
url : https://www.fmkorea.com//3507063645
url : https://www.fmkorea.com//3507063382
url : https://www.fmkorea.com//3507063645
url : https://www.fmkorea.com//3507063382
url : https://www.fmkorea.com//3507063081
url : https://www.fmkorea.com//3507062340
url : https://www.fmkorea.com//3507063000
url : https://www.fmkorea.com//3507062942
url : https://www.fmkorea.com//3507062688
url : https://www.fmkorea.com//350

url : https://www.fmkorea.com//3506984561
url : https://www.fmkorea.com//3506979601
url : https://www.fmkorea.com//3506978110
url : https://www.fmkorea.com//3506976637
url : https://www.fmkorea.com//3506974872
url : https://www.fmkorea.com//3506974443
url : https://www.fmkorea.com//3506970432
url : https://www.fmkorea.com//3506969669
url : https://www.fmkorea.com//3506967253
url : https://www.fmkorea.com//3506961512
url : https://www.fmkorea.com//3506942319
url : https://www.fmkorea.com//3506951827
url : https://www.fmkorea.com//3506957312
url : https://www.fmkorea.com//3506956526
url : https://www.fmkorea.com//3506949971
url : https://www.fmkorea.com//3506945089
url : https://www.fmkorea.com//3506942473
url : https://www.fmkorea.com//3506939221
url : https://www.fmkorea.com//3506937140
url : https://www.fmkorea.com//3506937619
url : https://www.fmkorea.com//3506937140
url : https://www.fmkorea.com//3506937619
url : https://www.fmkorea.com//3506935233
url : https://www.fmkorea.com//350

url : https://www.fmkorea.com//3506820551
url : https://www.fmkorea.com//3506820244
url : https://www.fmkorea.com//3506819790
url : https://www.fmkorea.com//3506819402
url : https://www.fmkorea.com//3506819402
url : https://www.fmkorea.com//3506818727
url : https://www.fmkorea.com//3506818591
url : https://www.fmkorea.com//3506817727
url : https://www.fmkorea.com//3506817182
url : https://www.fmkorea.com//3506817325
url : https://www.fmkorea.com//3506816237
url : https://www.fmkorea.com//3506817259
url : https://www.fmkorea.com//3506815988
url : https://www.fmkorea.com//3506815130
url : https://www.fmkorea.com//3506814953
url : https://www.fmkorea.com//3506814925
url : https://www.fmkorea.com//3506814573
url : https://www.fmkorea.com//3506814323
url : https://www.fmkorea.com//3506813714
url : https://www.fmkorea.com//3506813645
url : https://www.fmkorea.com//3506813482
url : https://www.fmkorea.com//3506813346
url : https://www.fmkorea.com//3506812204
url : https://www.fmkorea.com//350

url : https://www.fmkorea.com//3506744907
url : https://www.fmkorea.com//3506744340
url : https://www.fmkorea.com//3506743398
url : https://www.fmkorea.com//3506742165
url : https://www.fmkorea.com//3506741995
url : https://www.fmkorea.com//3506741783
url : https://www.fmkorea.com//3506741055
url : https://www.fmkorea.com//3506740443
url : https://www.fmkorea.com//3506741783
url : https://www.fmkorea.com//3506741055
url : https://www.fmkorea.com//3506740443
url : https://www.fmkorea.com//3506740078
url : https://www.fmkorea.com//3506738920
url : https://www.fmkorea.com//3506737946
url : https://www.fmkorea.com//3506738617
url : https://www.fmkorea.com//3506738361
url : https://www.fmkorea.com//3506737403
url : https://www.fmkorea.com//3506737680
url : https://www.fmkorea.com//3506737621
url : https://www.fmkorea.com//3506735536
url : https://www.fmkorea.com//3506735102
url : https://www.fmkorea.com//3506733681
url : https://www.fmkorea.com//3506733455
url : https://www.fmkorea.com//350

url : https://www.fmkorea.com//3506665874
url : https://www.fmkorea.com//3506665214
url : https://www.fmkorea.com//3506664926
url : https://www.fmkorea.com//3506664306
url : https://www.fmkorea.com//3506663621
url : https://www.fmkorea.com//3506663571
url : https://www.fmkorea.com//3506662970
url : https://www.fmkorea.com//3506662327
url : https://www.fmkorea.com//3506662156
url : https://www.fmkorea.com//3506661906
url : https://www.fmkorea.com//3506661667
url : https://www.fmkorea.com//3506651700
url : https://www.fmkorea.com//3506662327
url : https://www.fmkorea.com//3506662156
url : https://www.fmkorea.com//3506661906
url : https://www.fmkorea.com//3506661667
url : https://www.fmkorea.com//3506651700
url : https://www.fmkorea.com//3506661188
url : https://www.fmkorea.com//3506660501
url : https://www.fmkorea.com//3506659822
url : https://www.fmkorea.com//3506659322
url : https://www.fmkorea.com//3506657843
url : https://www.fmkorea.com//3506658412
url : https://www.fmkorea.com//350

In [30]:
print("Let's Test")

Let's Test


In [40]:
page_num = 7247

In [43]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
# df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['user_name'] = pd.Series(user2)
df['url'] = pd.Series(url2)
df['date'] = pd.Series(time2)
df['view_cnt'] = pd.Series(view_cnt2)
# df['like_cnt'] = pd.Series(like_cnt2)
df['user_text'] = pd.Series(text2)
df['com_cnt'] = pd.Series(com_cnt2)
df['com_user'] = pd.Series(com_user2)
df['com_text'] = pd.Series(com_text2)

df

,domain,title,user_name,url,date,view_cnt,user_text,com_cnt,com_user,com_text
0,fm,청문회 분명히 내일 좌파언론 중심으로 폄하 들어간다 ㅋㅋ,흐르는강의단풍잎,https://www.fmkorea.com//3528097412,2021-04-15,4346,내정간섭 타령하면서 ㅋㅋㅋ,3,"[zlzzlz, 4321, 레쓰]","[작성자 : zlzzlz :: 언론중심? ㄴㄴ 청와대 중심 ㅋㅋㅋㅋ, 작성자 : 4..."
1,fm,내가 모르겠어서 물어봄,bullwolf,https://www.fmkorea.com//3528092993,2021-04-15,242,혹시 육군도 핵관련 장비 만드는게 가능할꺼? ㅇㅇ핵 잠수함이나 핵 항공모함 같은거 ...,0,[],[]
2,fm,문재인 지지자들에게 궁금한 점,지우와알등이,https://www.fmkorea.com//3528092575,2021-04-15,260,재앙이가 하는 정책들 꼬라지 보고 자괴감 안 드냐?,4,"[닉넴엄따, 데상트코리아, 시베리아불곰, 유돈노]","[작성자 : 닉넴엄따 :: 잘하는 거래여 낄낄낄낄낄, 작성자 : 데상트코리아 :: ..."
3,fm,희망이 없으니 애도 못낳고 결혼도 못하는거임 출산율 올리려면 희망을 만들어야 하는데,월간발정호소당,https://www.fmkorea.com//3528091987,2021-04-15,72,문메갈은 절망 분노만 만듬나도 다 포기했음 ㅠㅠㅠ 삶의 희망이 안보여 ㅠㅠ,0,[],[]
4,fm,정신 못차리네 이새기들 ㅋㅋㅋㅋㅋㅋㅋㅋ??,길거리투팍,https://www.fmkorea.com//3528088980,2021-04-15,8347,경향? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ시발년들아 너넨 확정이야 개새끼들아 ㅋㅋ...,22,"[시콘, 힘내보자, ㅂㅊ, bullwolf, 최재형, VIBE, 길거리투팍, cob...","[작성자 : 시콘 :: ㅋㅋㅋㅋ, 작성자 : 힘내보자 :: 아니 자기들 법안발의 안..."
...,...,...,...,...,...,...,...,...,...,...
52529,fm,이건 보궐 이겨도 문제네,가나둘셋,https://www.fmkorea.com//3506619553,2021-04-07,79,1년짜리 보궐가지고도 180석 거대여당이 저ㅈㄹ인데 대선은 볼만하겠네. 여태까지 보...,0,[],[]
52530,fm,이제 12시 넘었으니까 다들 워딩 조심하자구,마리스풍선,https://www.fmkorea.com//3506619253,2021-04-07,412,ㅈㄴ 짜증나는건 알지만 일단 워딩은 조심히 하자,1,[돌로레스무왕가],[작성자 : 돌로레스무왕가 :: 오늘만 로갓해야징]
52531,fm,ㄹㅇ ㅋㅋ,K-새끼,https://www.fmkorea.com//3506619240,2021-04-07,21,ㄹㅇ ㅋㅋㅋㅋ,0,[],[]
52532,fm,ㄹㅇㅋㅋ,공산당,https://www.fmkorea.com//3506618335,2021-04-07,27,ㄹㅇㅋㅋ,0,[],[]


In [44]:
for i in range(len(df)) :
    if len(df['user_text'][i]) < 5 :
        df['user_text'][i] = df['title'][i]

In [45]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 171616.9 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\Qscar KIM\Desktop\daily\2021-04-07-2021-04-15\\fm.csv 


SystemExit: 프로그램을 종료합니다.

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 